In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("punto_de_pedido.xlsx")

In [3]:
def column_with_desired_data(raw_dataframe, art, descrip, vpd):
    result_df = raw_dataframe[[art,descrip,vpd]]
    return result_df

In [4]:
df.columns

Index(['ARTÍCULO', 'ARTÍCULO COMPAÑÍA', 'DESCRIPCIÓN', 'BULTOS X PISO',
       'BULTOS X PALLETS', 'STOCK', 'VTA. PROMEDIO', 'COEF. ACCIÓN',
       'VTA. CALCULADA', 'DÍAS DE STOCK', 'POLITICA', 'YA PEDIDO',
       'FECHA ÚLTIMO PEDIDO', 'DÍAS VENTANA', 'DÍAS PARA ARRIBO',
       'STOCK TEÓRICO AL ARRIBO', 'MIN. NECESARIO', 'MIN. A PEDIR',
       'MIN. PALLETS A PEDIR', 'PEDIR?'],
      dtype='object')

In [10]:
def sorted_and_cumulative_df(df, column_to_sort):
    df = df.sort_values(column_to_sort, ascending=False)
    df['Acumulado VPD'] = df[column_to_sort].cumsum()
    return df   

In [30]:
def vpd_weight(df, cum_column):
    total_vpd = list(df[cum_column])[-1]
    df['Porcentual acum. VPD'] = df[cum_column].apply(lambda x: (x/total_vpd)*100)
    return df

In [52]:
def compare_with_limits(x,a,b):
    if x <= a:
        return 'A'
    elif  x <= b and x > a:
        return 'B'
    else:
        return 'C'

In [53]:
def abc_clasification(df,a=80, b=95):
    df['Categoría'] = df['Porcentual acum. VPD'].apply(lambda x: compare_with_limits(x,a,b))
    return df

In [5]:
df = column_with_desired_data(df,'ARTÍCULO','DESCRIPCIÓN','VTA. PROMEDIO')

In [56]:
df = sorted_and_cumulative_df(df,'VTA. PROMEDIO')

In [54]:
vpd_weight(df,'Acumulado VPD')

,ARTÍCULO,DESCRIPCIÓN,VTA. PROMEDIO,Acumulado VPD,Porcentual acum. VPD,Categoría
34,7038,BRAHMA - X12 1000RET#,1056.42,1056.42,31.013164,A
44,7634,BRAHMA - 4X6 LAT 473,289.64,1346.06,39.516082,A
223,25668,PEPSI NF PET X6 500CC,125.88,1471.94,43.211522,A
30,7026,QUILMES - X12 1000RET,102.76,1574.70,46.228232,A
219,25644,7-UP NF PET X6 500CC,99.96,1674.66,49.162743,A
...,...,...,...,...,...,...
196,24968,SIDRA 1930 DEMI SEC X6 750CC,0.00,3406.36,100.000000,C
75,15159,PATA IPA SAUCO 30L KEG,0.00,3406.36,100.000000,C
285,77812,CBSE HIERBAS SERRANAS X12 500GR,0.00,3406.36,100.000000,C
40,7475,PASO DLT POMELO * X6 PET1500#,0.00,3406.36,100.000000,C


In [58]:
abc_clasification(df).head(80)

,ARTÍCULO,DESCRIPCIÓN,VTA. PROMEDIO,Acumulado VPD,Porcentual acum. VPD,Categoría
34,7038,BRAHMA - X12 1000RET#,1056.42,1056.42,31.013164,A
44,7634,BRAHMA - 4X6 LAT 473,289.64,1346.06,39.516082,A
223,25668,PEPSI NF PET X6 500CC,125.88,1471.94,43.211522,A
30,7026,QUILMES - X12 1000RET,102.76,1574.70,46.228232,A
219,25644,7-UP NF PET X6 500CC,99.96,1674.66,49.162743,A
...,...,...,...,...,...,...
202,25035,AW POMELO SIN AZ AG X6 2250CC,6.68,3041.49,89.288566,B
77,16371,RED BULL CAN 6X4 250,6.59,3048.08,89.482028,B
88,18354,PATAGONIA IPA OW X6 730,6.56,3054.64,89.674609,B
180,24683,BUDWEISER CAN 4X6 473CC TRANSP,6.56,3061.20,89.867190,B


In [51]:
df

,ARTÍCULO,DESCRIPCIÓN,VTA. PROMEDIO,Acumulado VPD,Porcentual acum. VPD,Categoría
34,7038,BRAHMA - X12 1000RET#,1056.42,1056.42,31.013164,A
44,7634,BRAHMA - 4X6 LAT 473,289.64,1346.06,39.516082,A
223,25668,PEPSI NF PET X6 500CC,125.88,1471.94,43.211522,A
30,7026,QUILMES - X12 1000RET,102.76,1574.70,46.228232,A
219,25644,7-UP NF PET X6 500CC,99.96,1674.66,49.162743,A
...,...,...,...,...,...,...
196,24968,SIDRA 1930 DEMI SEC X6 750CC,0.00,3406.36,100.000000,C
75,15159,PATA IPA SAUCO 30L KEG,0.00,3406.36,100.000000,C
285,77812,CBSE HIERBAS SERRANAS X12 500GR,0.00,3406.36,100.000000,C
40,7475,PASO DLT POMELO * X6 PET1500#,0.00,3406.36,100.000000,C
